# Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

Links:

[[all_installetion_points_parquet]]<br>
[[companies_st_partner]]<br>
[[companies_dir_partner]]<br>
[[billing_orders_devices_st_partner]]<br>
[[cameras_st_partner]]<br>
[[cameras_dir_partner]]<br>
[[intercoms_dir_asgard]]

# Creating a table

In [2]:
query_text = """--sql
    with all_installetion_points_parquet AS (
    SELECT
        `address_uuid`,
        `city`,
        `country`,
        `created_at`,
        `full_address`,
        `installation_point_id`,
        `parent_uuid`,
        `region`,
        `report_date`,
    FROM db1.`all_installetion_points_parquet_ch`
    ),
    companies_st_partner AS (
    SELECT
        `report_date`,
        companies_st_partner.`partner_uuid` as partner_uuid,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk`,
        `registration_date`,
        `tin`,
        `kpp`
    FROM db1.`companies_st_partner_ch` AS companies_st_partner 
    LEFT JOIN db1.`companies_dir_partner_ch` AS companies_dir_partner 
        ON  companies_st_partner.partner_uuid = companies_dir_partner.partner_uuid 
    ),
    billing_orders_devices_dir_partner AS (
    SELECT
        `billing_account_id`,
        `cost`,
        `count`,
        toDate(`created_at`) as report_date,
        `device_type`,
        `device_uuid`,
        `partner_uuid`,
        `service`,
        `state`,
        `total`
    FROM db1.`billing_orders_devices_st_partner_ch`
    ),
    cameras_st_partner AS (
    SELECT
        `report_date`,
        cameras_st_partner.`service_partner_uuid` AS service_partner_uuid,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        if(`motherboard_id`!= '',`motherboard_id`,`serial_number`) AS device_serial_number,
        if(cameras_st_partner.`intercom_uuid` != '',cameras_st_partner.`intercom_uuid` ,cameras_st_partner.`camera_uuid`) AS device_uuid,
        if(cameras_st_partner.`intercom_uuid` != '', 'intercom', if(`foreign_camera` = 1,'camera_foreign','camera')) AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        `foreign_camera`
    FROM db1.`cameras_st_partner_ch` AS cameras_st_partner
    LEFT JOIN db1.`cameras_dir_partner_ch` AS cameras_dir_partner 
        ON cameras_st_partner.camera_uuid = cameras_dir_partner.camera_uuid
    LEFT JOIN db1.`intercoms_dir_asgard_ch` AS intercoms_dir_asgard 
        ON cameras_st_partner.intercom_uuid = intercoms_dir_asgard.intercom_uuid
    )
    --
    SELECT 
        DISTINCT
        cameras_st_partner.`report_date` AS report_date,
        `service_partner_uuid`,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        cameras_st_partner.`device_uuid` AS device_uuid,
        device_serial_number,
        CASE
            WHEN cameras_st_partner.device_type = 'intercom' THEN 'Домофон'
            WHEN cameras_st_partner.device_type = 'camera' THEN 'Камера IZI'
            WHEN cameras_st_partner.device_type = 'camera_foreign' THEN 'Сторонняя камера'
            ELSE NULL
        END AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `foreign_camera`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        IF(`service`!='', `service`, 'Нет услуг') AS `service`,
        `total`,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk`,
        `registration_date`,
        `tin`,
        `kpp`,
        -- `address_uuid`,
        `city`,
        `country`,
        -- `created_at`,
        `full_address`,
        `parent_uuid`,
        `region`,
    FROM cameras_st_partner
    LEFT JOIN billing_orders_devices_dir_partner
        ON billing_orders_devices_dir_partner.report_date = cameras_st_partner.report_date    
        AND billing_orders_devices_dir_partner.device_uuid = cameras_st_partner.device_uuid
    LEFT JOIN  companies_st_partner
        ON companies_st_partner.report_date = cameras_st_partner.report_date    
        AND companies_st_partner.partner_uuid = cameras_st_partner.partner_uuid
    LEFT JOIN all_installetion_points_parquet
        ON all_installetion_points_parquet.report_date = cameras_st_partner.report_date    
        AND all_installetion_points_parquet.installation_point_id = cameras_st_partner.installation_point_id
    limit 10
    -- WHERE 
        -- NOT(service="pro" and pro_subs=1 and billing_pro=0)
        -- service not in ["pro_kz","archive_3_free","developer","developer_trial","developer_free","archive_3_kz","archive_5_kz","mobile_pack","mobile_pack_kz"]
        -- AND billing_orders_devices_dir_partner.report_date != CurrentUtcDate()
    """
ch.query_run(query_text)

,report_date,service_partner_uuid,partner_uuid,device_uuid,device_serial_number,device_type,installation_point_id,camera_dvr_depth,foreign_camera,archive_from_partner,...,company_name,partner_lk,registration_date,tin,kpp,city,country,full_address,parent_uuid,region
0,2024-02-01,,af8b5bf4-0108-4a0c-acc2-096f94959fb3,884cabd1-0fff-4210-a801-e02c11090388,1835100129,Домофон,0,0,0,,...,ТОО РИТМ ПРО,120074,2019-04-05,,,,,,,
1,2024-02-01,963e8bcc-c6c3-46ee-958f-412c226ac672,963e8bcc-c6c3-46ee-958f-412c226ac672,221d8b57-d74f-49c8-9422-38b2d59a2597,DS-2CDVT-NONE-S0120160606AAWR608590451,Домофон,44,0,0,,...,ООО Bид Плюс ПРО,120696,2019-11-20,7422048677,741301001,,,,,
2,2024-02-01,11dad87f-455e-4484-9d85-e38ba9a6f7f0,11dad87f-455e-4484-9d85-e38ba9a6f7f0,d6b40231-558b-4d44-ba10-b0b481318f6e,1809100050,Домофон,164327,3,0,1,...,ООО Ктототам,123047,2021-02-27,,,,,,,
3,2024-02-01,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,688fa517-a819-4157-bd79-6d237ef0936b,1802100009,Домофон,47,0,0,,...,Спутник,120012,2018-10-30,7731323200,111111111,,,,,
4,2024-02-01,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,8657344b-e9ee-48bc-9f2c-45d11a01d71e,1809100122,Домофон,59,0,0,,...,Спутник,120012,2018-10-30,7731323200,111111111,,,,,
5,2024-02-01,db61f873-dcf7-4678-aec1-6b93f54ddbb2,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,d8d45214-6a8f-4240-a6b7-a8a37fef55c1,1809100093,Домофон,0,0,0,,...,Спутник,120012,2018-10-30,7731323200,111111111,,,,,
6,2024-02-01,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,df492dfe-8237-4a57-9090-b7175f0db0d1,1843100134,Домофон,68,0,0,,...,Спутник,120012,2018-10-30,7731323200,111111111,,,,,
7,2024-02-01,,c4cea074-f4b6-47ba-af6c-97422bca530f,9898dea4-6c63-4339-9201-bf7a274624bc,1845100138,Домофон,73,0,0,,...,MIRCOD,120061,2019-02-04,1655359346,165501001,,,,,
8,2024-02-01,,94d9ac68-e1b5-4ebe-a902-b78e386cef7d,c4fcfc50-27f9-4513-b651-68e9bcf11ee5,1845100135,Домофон,0,0,0,,...,,120941,2020-02-06,0276924627,027601001,,,,,
9,2024-02-01,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,b1782e4f-9198-49d1-b5aa-7bdba9c87d21,ab0bb204-f2a3-46e8-bc7e-c65ba68cdb05,1848100193,Домофон,0,0,0,,...,Спутник,120012,2018-10-30,7731323200,111111111,,,,,


In [62]:
query_text = """--sql
    CREATE TABLE db1.t_device_billins
    (
        `report_date` Date,
        `service_partner_uuid` String,
        `partner_uuid` String,
        `device_uuid` String,
        `device_serial_number` String,
        `device_type` LowCardinality(String),
        `installation_point_id` UInt32,
        `camera_dvr_depth` UInt8,
        `foreign_camera` UInt8,
        `archive_from_partner` LowCardinality(String),
        `included_by` String,
        `included_at` DateTime,
        `disabled_by` String,
        `disabled_at` DateTime,
        `service` LowCardinality(String),
        `total` Float32,
        `is_blocked` UInt8,
        `pro_subs` UInt8,
        `enterprise_subs` UInt8,
        `billing_pro` UInt8,
        `enterprise_not_paid` UInt8,
        `enterprise_test` UInt8,
        `balance` Float64,
        `kz_pro`UInt8,
        `company_name` String,
        `partner_lk` UInt32,
        `registration_date` Date,
        `tin` String,
        `kpp` String,
        `city` String,
        `country` LowCardinality(String),
        `full_address` String,
        `parent_uuid` String,
        `region` LowCardinality(String)
    )

    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

""


In [ ]:
query_text = """--sql
    CREATE TABLE db1.t_device_billins
    (
        `report_date` Date,
        `service_partner_uuid` String,
        `partner_uuid` String,
        `device_uuid` String,
        `device_serial_number` String,
        `device_type` LowCardinality(String),
        `installation_point_id` UInt32,
        `camera_dvr_depth` UInt8,
        `foreign_camera` UInt8,
        `archive_from_partner` LowCardinality(String),
        `included_by` String,
        `included_at` DateTime,
        `disabled_by` String,
        `disabled_at` DateTime,
        `service` LowCardinality(String),
        `total` Float32,
        `is_blocked` UInt8,
        `pro_subs` UInt8,
        `enterprise_subs` UInt8,
        `billing_pro` UInt8,
        `enterprise_not_paid` UInt8,
        `enterprise_test` UInt8,
        `balance` Float64,
        `kz_pro`UInt8,
        `company_name` String,
        `partner_lk` UInt32,
        `registration_date` Date,
        `tin` String,
        `kpp` String,
        `city` String,
        `country` LowCardinality(String),
        `full_address` String,
        `parent_uuid` String,
        `region` LowCardinality(String)
    )

    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

In [ ]:
query_text = """
    CREATE MATERIALIZED VIEW db1.t_device_billins_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 25 MINUTE TO db1.t_device_billins AS
    WITH all_installetion_points_parquet AS (
    SELECT
        `address_uuid`,
        `city`,
        `country`,
        `created_at`,
        `full_address`,
        `installation_point_id`,
        `parent_uuid`,
        `region`,
        `report_date`,
    FROM db1.`all_installetion_points_parquet_ch`
    ),
    companies_st_partner AS (
    SELECT
        `report_date`,
        companies_st_partner.`partner_uuid` as partner_uuid,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk`,
        `registration_date`,
        `tin`,
        `kpp`
    FROM db1.`companies_st_partner_ch` AS companies_st_partner 
    LEFT JOIN db1.`companies_dir_partner_ch` AS companies_dir_partner 
        ON  companies_st_partner.partner_uuid = companies_dir_partner.partner_uuid 
    ),
    billing_orders_devices_dir_partner AS (
    SELECT
        `billing_account_id`,
        `cost`,
        `count`,
        toDate(`created_at`) as report_date,
        `device_type`,
        `device_uuid`,
        `partner_uuid`,
        `service`,
        `state`,
        `total`
    FROM db1.`billing_orders_devices_st_partner_ch`
    ),
    cameras_st_partner AS (
    SELECT
        `report_date`,
        cameras_st_partner.`service_partner_uuid` AS service_partner_uuid,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        if(`motherboard_id`!= '',`motherboard_id`,`serial_number`) AS device_serial_number,
        if(cameras_st_partner.`intercom_uuid` != '',cameras_st_partner.`intercom_uuid` ,cameras_st_partner.`camera_uuid`) AS device_uuid,
        if(cameras_st_partner.`intercom_uuid` != '', 'intercom', if(`foreign_camera` = 1,'camera_foreign','camera')) AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        `foreign_camera`
    FROM db1.`cameras_st_partner_ch` AS cameras_st_partner
    LEFT JOIN db1.`cameras_dir_partner_ch` AS cameras_dir_partner 
        ON cameras_st_partner.camera_uuid = cameras_dir_partner.camera_uuid
    LEFT JOIN db1.`intercoms_dir_asgard_ch` AS intercoms_dir_asgard 
        ON cameras_st_partner.intercom_uuid = intercoms_dir_asgard.intercom_uuid
    )
    --
    SELECT 
        DISTINCT
        cameras_st_partner.`report_date` AS report_date,
        `service_partner_uuid`,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        cameras_st_partner.`device_uuid` AS device_uuid,
        device_serial_number,
        CASE
            WHEN cameras_st_partner.device_type = 'intercom' THEN 'Домофон'
            WHEN cameras_st_partner.device_type = 'camera' THEN 'Камера IZI'
            WHEN cameras_st_partner.device_type = 'camera_foreign' THEN 'Сторонняя камера'
            ELSE NULL
        END AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `foreign_camera`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        IF(`service`!='', `service`, 'Нет услуг') AS `service`,
        `total`,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk`,
        `registration_date`,
        `tin`,
        `kpp`,
        -- `address_uuid`,
        `city`,
        `country`,
        -- `created_at`,
        `full_address`,
        `parent_uuid`,
        `region`,
    FROM cameras_st_partner
    LEFT JOIN billing_orders_devices_dir_partner
        ON billing_orders_devices_dir_partner.report_date = cameras_st_partner.report_date    
        AND billing_orders_devices_dir_partner.device_uuid = cameras_st_partner.device_uuid
    LEFT JOIN  companies_st_partner
        ON companies_st_partner.report_date = cameras_st_partner.report_date    
        AND companies_st_partner.partner_uuid = cameras_st_partner.partner_uuid
    LEFT JOIN all_installetion_points_parquet
        ON all_installetion_points_parquet.report_date = cameras_st_partner.report_date    
        AND all_installetion_points_parquet.installation_point_id = cameras_st_partner.installation_point_id
    """
ch.query_run(query_text)

,elapsed_ns,query_id
0,201424674,a0521a0d-b2f9-4f06-ac96-fa1f605527d9


In [68]:
query_text = """ 
DROP TABLE db1.t_device_billins_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,1219411,3fab9b51-681d-4f26-93ae-66058fab52e5


In [65]:
query_text = """
SYSTEM REFRESH VIEW db1.t_device_billins_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,860755,72cf2142-db94-4b96-9bb1-6895964d34e6


In [66]:
query_text = """ 
SELECT
    *
FROM db1.t_device_billins
limit 10
"""

ch.query_run(query_text)

""
